In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# Load fine-tuned model
model_name = "m-newhauser/distilbert-political-tweets"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

C:\Users\Filippo Corti\miniconda3\envs\SocialMediaMining\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [13]:
text = "President Trump is the best America has ever had"
text2 = "President Trump is the worst America has ever had"

result = classifier(text)
result2 = classifier(text2)

print(result)
print(result2)

[{'label': 'Republican', 'score': 0.9959179759025574}]
[{'label': 'Republican', 'score': 0.9999874830245972}]


In [21]:
text = "President Trump is the worst America has ever had"

# Tokenize
inputs = tokenizer(text, return_tensors="pt")
labels = torch.tensor([0])
outputs = model(**inputs, labels=labels)
loss, logits = outputs[:2]
print(logits.softmax(dim=-1)[0].tolist()) # Result in [Left, Center, Right]

[0.9999874830245972, 1.256377527170116e-05]


In [ ]:
# The Issue is that it's trained to recognize political bias in news. It needs fine-tuning on small texts
# (politicalBiasBERT)

# About m-newhauser/distilbert-political-tweets > Same issue apparently

# I should probably match it with sentiment analysis tool (or

In [1]:
print(torch.cuda.is_available())

NameError: name 'torch' is not defined